## IMPORTING THE DATA

In [ ]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os. chdir('drive')
os. chdir('MyDrive')

In [ ]:
os. chdir('dataset')

In [ ]:
train = pd.read_csv('train_tweet.csv')
train1 = pd.read_csv('neg_words.csv')
train2 = pd.read_csv('neg_phrase.csv')
train3 = pd.read_csv('pos_words.csv')
train4 = pd.read_csv('pos_phrase.csv')
test = pd.read_csv('test_tweets.csv')

print(train.shape)
print(test.shape)

(31962, 3)
(17197, 2)


In [ ]:
train1.head()

sss  1
0     2-faces  1
1    abnormal  1
2     abolish  1
3  abominable  1
4  abominably  1

In [ ]:
train2.head()

sss  1
0  it's so laddish and juvenile , only teenage bo...  1
1  exploitative and largely devoid of the depth o...  1
2  [garbus] discards the potential for pathologic...  1
3  a visually flashy but narratively opaque and e...  1
4  the story is also as unoriginal as they come ,...  1

In [ ]:
train4.head()

sss  0
0  the gorgeously elaborate continuation of " the...  0
1                     effective but too-tepid biopic  0
2  if you sometimes like to go to the movies to h...  0
3  emerges as something rare , an issue movie tha...  0
4  the film provides some great insight into the ...  0

In [ ]:
train3.head()

sss  0
0      abound  0
1     abounds  0
2   abundance  0
3    abundant  0
4  accessable  0

In [ ]:
IMDBtrain=pd.read_csv('IMDB_dataset.csv')

In [ ]:
IMDBtrain.head()

review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive

In [ ]:
def label(element):
    if element=="positive":
        return 0
    elif element=="negative":
        return 1
    else:
        return element

In [ ]:
IMDBtrain['sentiment']=IMDBtrain['sentiment'].apply(label)

In [ ]:
IMDBtrain.size

100000

In [ ]:
IMDBtrain['review'] = IMDBtrain['review'].str.replace(r'<[^<>]*>', '', regex=True)

In [ ]:
IMDBtrain.head()

review  sentiment
0  One of the other reviewers has mentioned that ...          0
1  A wonderful little production. The filming tec...          0
2  I thought this was a wonderful way to spend ti...          0
3  Basically there's a family where a little boy ...          1
4  Petter Mattei's "Love in the Time of Money" is...          0

In [ ]:
train.head()

id  label                                              tweet
0   1      0   @user when a father is dysfunctional and is s...
1   2      0  @user @user thanks for #lyft credit i can't us...
2   3      0                                bihday your majesty
3   4      0  #model   i love u take with u all the time in ...
4   5      0             factsguide: society now    #motivation

In [ ]:
test.head()

id                                              tweet
0  31963  #studiolife #aislife #requires #passion #dedic...
1  31964   @user #white #supremacists want everyone to s...
2  31965  safe ways to heal your #acne!!    #altwaystohe...
3  31966  is the hp and the cursed child book up for res...
4  31967    3rd #bihday to my amazing, hilarious #nephew...

In [ ]:
train.isnull().any()
test.isnull().any()

id       False
tweet    False
dtype: bool

In [ ]:
# checking out the negative comments from the train set

train[train['label'] == 0].head(10)

id  label                                              tweet
0   1      0   @user when a father is dysfunctional and is s...
1   2      0  @user @user thanks for #lyft credit i can't us...
2   3      0                                bihday your majesty
3   4      0  #model   i love u take with u all the time in ...
4   5      0             factsguide: society now    #motivation
5   6      0  [2/2] huge fan fare and big talking before the...
6   7      0   @user camping tomorrow @user @user @user @use...
7   8      0  the next school year is the year for exams.ð...
8   9      0  we won!!! love the land!!! #allin #cavs #champ...
9  10      0   @user @user welcome here !  i'm   it's so #gr...

In [ ]:
# checking out the postive comments from the train set 

train[train['label'] == 1].head(10)

id  label                                              tweet
13    14      1  @user #cnn calls #michigan middle school 'buil...
14    15      1  no comment!  in #australia   #opkillingbay #se...
17    18      1                             retweet if you agree! 
23    24      1    @user @user lumpy says i am a . prove it lumpy.
34    35      1  it's unbelievable that in the 21st century we'...
56    57      1            @user lets fight against  #love #peace 
68    69      1  ð©the white establishment can't have blk fol...
77    78      1  @user hey, white people: you can call people '...
82    83      1  how the #altright uses  &amp; insecurity to lu...
111  112      1  @user i'm not interested in a #linguistics tha...

## VISUALIZATION

In [ ]:
train['label'].value_counts().plot.bar(color = 'pink', figsize = (6, 4))

In [ ]:
# checking the distribution of tweets in the data

length_train = train['tweet'].str.len().plot.hist(color = 'pink', figsize = (6, 4))
length_test = test['tweet'].str.len().plot.hist(color = 'orange', figsize = (6, 4))

In [ ]:
# adding a column to represent the length of the tweet

train['len'] = train['tweet'].str.len()
test['len'] = test['tweet'].str.len()

train.head(10)

In [ ]:

train.groupby('label').describe()

In [ ]:
train.groupby('len').mean()['label'].plot.hist(color = 'black', figsize = (6, 4),)
plt.title('variation of length')
plt.xlabel('Length')
plt.show()

## PREPROCESSING

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


cv = CountVectorizer(stop_words = 'english',max_features=20000)
words = cv.fit_transform(train.tweet)
sum_words = words.sum(axis=0)

words_freq = [(word, sum_words[0, i]) for word, i in cv.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)

frequency = pd.DataFrame(words_freq, columns=['word', 'freq'])

frequency.head(30).plot(x='word', y='freq', kind='bar', figsize=(15, 7), color = 'blue')
plt.title("Most Frequently Occuring Words - Top 30")
# print(cv.vocabulary_)

In [ ]:
from wordcloud import WordCloud

wordcloud = WordCloud(background_color = 'white', width = 1000, height = 1000).generate_from_frequencies(dict(words_freq))

plt.figure(figsize=(10,8))
plt.imshow(wordcloud)
plt.title("WordCloud - Vocabulary from Reviews", fontsize = 22)

In [ ]:
normal_words =' '.join([text for text in train['tweet'][train['label'] == 0]])

wordcloud = WordCloud(width=800, height=500, random_state = 0, max_font_size = 110).generate(normal_words)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.title('The Neutral Words')
plt.show()


In [ ]:
negative_words =' '.join([text for text in train['tweet'][train['label'] == 1]])

wordcloud = WordCloud(background_color = 'cyan', width=800, height=500, random_state = 0, max_font_size = 110).generate(negative_words)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.title('The Negative Words')
plt.show()


In [ ]:
# collecting the hashtags

def hashtag_extract(x):
    hashtags = []
    
    for i in x:
        ht = re.findall(r"#(\w+)", i)
        hashtags.append(ht)

    return hashtags

In [ ]:
# extracting hashtags from non racist/sexist tweets
HT_regular = hashtag_extract(train['tweet'][train['label'] == 0])

# extracting hashtags from racist/sexist tweets
HT_negative = hashtag_extract(train['tweet'][train['label'] == 1])

# unnesting list
HT_regular = sum(HT_regular,[])
HT_negative = sum(HT_negative,[])

In [ ]:
a = nltk.FreqDist(HT_regular)
d = pd.DataFrame({'Hashtag': list(a.keys()),
                  'Count': list(a.values())})

# selecting top 20 most frequent hashtags     
d = d.nlargest(columns="Count", n = 20) 
plt.figure(figsize=(16,5))
ax = sns.barplot(data=d, x= "Hashtag", y = "Count")
ax.set(ylabel = 'Count')
plt.show()

In [ ]:
a = nltk.FreqDist(HT_negative)
d = pd.DataFrame({'Hashtag': list(a.keys()),
                  'Count': list(a.values())})

# selecting top 20 most frequent hashtags     
d = d.nlargest(columns="Count", n = 20) 
plt.figure(figsize=(16,5))
ax = sns.barplot(data=d, x= "Hashtag", y = "Count")
ax.set(ylabel = 'Count')
plt.show()

In [ ]:
# tokenizing the words present in the training set
tokenized_tweet = train['tweet'].apply(lambda x: x.split()) 

# importing gensim
import gensim

# creating a word to vector model
model_w2v = gensim.models.Word2Vec(
            tokenized_tweet,
            vector_size=200, # desired no. of features/independent variables 
            window=5, # context window size
            min_count=4, 
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 2, # no.of cores
            seed = 34)

model_w2v.train(tokenized_tweet, total_examples= len(train['tweet']), epochs=20)


In [ ]:
model_w2v.wv.most_similar(positive = "dinner")

In [ ]:
model_w2v.wv.most_similar(positive = "cancer")

In [ ]:
model_w2v.wv.most_similar(positive = "apple")

In [ ]:
model_w2v.wv.most_similar(negative = "hate")

## Test and Training set Making 

In [ ]:
# removing unwanted patterns from the data

import re
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
train_corpus = []

for i in range(0, 31962):
  review = re.sub('[^a-zA-Z]', ' ', train['tweet'][i])
  review = review.lower()
  review = review.split()
  
  ps = PorterStemmer()
  
  # stemming
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  
  # joining them back with space
  review = ' '.join(review)
  train_corpus.append(review)

In [ ]:
test_corpus = []

for i in range(0, 17197):
  review = re.sub('[^a-zA-Z]', ' ', test['tweet'][i])
  review = review.lower()
  review = review.split()
  
  ps = PorterStemmer()
  
  # stemming
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  
  # joining them back with space
  review = ' '.join(review)
  test_corpus.append(review)

KeyboardInterrupt: ignored

In [ ]:
IMDBtrain.shape

(50000, 2)

In [ ]:
IMDB_corpus = []

for i in range(0, 50000):
  review = re.sub('[^a-zA-Z]', ' ', IMDBtrain['review'][i])
  review = review.lower()
  review = review.split()
  
  ps = PorterStemmer()
  
  # stemming
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  
  # joining them back with space
  review = ' '.join(review)
  IMDB_corpus.append(review)

In [ ]:
ytlist=["good","bad","distrack"]
yt1=[]
for i in range(0, len(ytlist)):
  review = re.sub('[^a-zA-Z]', ' ', ytlist[i])
  review = review.lower()
  review = review.split()
  
  ps = PorterStemmer()
  
  # stemming
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  
  # joining them back with space
  review = ' '.join(review)
  yt1.append(review)

In [ ]:
def preprocessing(ytlist):
    yt1=[]
    for i in range(0, len(ytlist)):
        review = re.sub('[^a-zA-Z]', ' ', ytlist[i])
        review = review.lower()
        review = review.split()
        ps = PorterStemmer()
        review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
        review = ' '.join(review)
        yt1.append(review)
    return yt1

In [ ]:
train11=preprocessing(train1['sss'])
train22=preprocessing(train2['sss'])
train33=preprocessing(train3['sss'])
train44=preprocessing(train4['sss'])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 10000)
cv.fit_transform(train11+train22+train33+train44)

<17447x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 114746 stored elements in Compressed Sparse Row format>

In [ ]:
print((cv.vocabulary_))

{'face': 2634, 'abomin': 12, 'abort': 13, 'abras': 15, 'abrupt': 17, 'abruptli': 18, 'absenc': 19, 'absent': 20, 'mind': 4765, 'absurd': 25, 'absurdli': 27, 'abus': 30, 'abysm': 31, 'abyss': 32, 'accident': 40, 'accus': 52, 'acerb': 54, 'ach': 55, 'acrid': 63, 'acrimoni': 64, 'adam': 78, 'addict': 81, 'admonish': 94, 'adulter': 107, 'adulteri': 108, 'adversari': 113, 'advers': 112, 'afflict': 126, 'affront': 130, 'afraid': 134, 'aggrav': 148, 'aggress': 149, 'aggriev': 150, 'agoni': 155, 'agon': 154, 'aground': 157, 'ail': 162, 'ailment': 163, 'aimless': 165, 'alarm': 177, 'alarmingli': 178, 'alien': 190, 'alleg': 193, 'allergi': 198, 'aloof': 210, 'ambigu': 230, 'ambival': 233, 'amiss': 246, 'anarch': 260, 'anarchist': 262, 'anarchi': 261, 'anem': 269, 'anger': 272, 'angri': 274, 'anguish': 276, 'annihil': 284, 'annoy': 285, 'annoyingli': 286, 'anomali': 287, 'antagon': 293, 'antagonist': 294, 'anti': 297, 'american': 238, 'isra': 3998, 'occup': 5173, 'semit': 6883, 'social': 7433, 'u

In [ ]:
def changetonum(word):
    try:
#         print(cv.vocabulary_[word])
        return cv.vocabulary_[word]
    except:
        return -1

In [ ]:

    
train_corpus1=[]
for i in train_corpus:
    lq=[]
    for j in i.split():
        if(changetonum(j)!=-1):
            lq.append(changetonum(j))
    train_corpus1.append(lq)
print(train_corpus1[0])

[9180, 2437, 2093, 7598, 2019, 4336, 2093, 7390]


In [ ]:
test_corpus1=[]
for i in test_corpus:
    lq=[]
    for j in i.split():
        if(changetonum(j)!=-1):
            lq.append(changetonum(j))
    test_corpus1.append(lq)
print(test_corpus1[0])

[8223, 7136, 6158, 1765, 2532]


In [ ]:
IMDB_corpus1=[]
for i in IMDB_corpus:
    lq=[]
    for j in i.split():
        if(changetonum(j)!=-1):
            lq.append(changetonum(j))
    IMDB_corpus1.append(lq)
print(IMDB_corpus1[0])

[5441, 6503, 4897, 8571, 2597, 3726, 6550, 2698, 3525, 2940, 7862, 7508, 995, 8262, 6768, 8485, 6918, 6550, 8742, 3309, 8092, 7011, 2796, 3588, 7927, 7011, 6123, 6132, 6366, 2335, 6928, 8485, 1374, 8376, 8742, 1087, 3285, 4829, 7403, 3014, 4716, 1358, 2736, 6849, 6032, 1205, 3290, 3132, 2784, 3656, 145, 2480, 1358, 3712, 4751, 3199, 1329, 4145, 4120, 1915, 7398, 1909, 5266, 2828, 524, 8763, 6749, 4714, 341, 7011, 2348, 2789, 3317, 7011, 1878, 3043, 6009, 5801, 5577, 4717, 481, 3043, 1255, 3043, 6602, 4914, 395, 2940, 2597, 2676, 6748, 7508, 5207, 7641, 6749, 6282, 8571, 2072, 7751, 3356, 3656, 4517, 3391, 8485, 8485, 3998, 3456, 4323, 5477, 3259, 524, 8617, 4760, 4938, 1372, 8118, 6032, 2348, 4394, 7487, 7103, 6032, 2733, 8571, 4830, 645, 1494, 8180, 8472, 3259, 7990, 1882, 7038]


In [ ]:
yt_corpus1=[]
for i in yt1:
    lq=[]
    for j in i.split():
        if(changetonum(j)!=-1):
            lq.append(changetonum(j))
    yt_corpus1.append(lq)
print(yt_corpus1[0])

[2955]


In [ ]:
def preprocessing2(yt1):
    yt_corpus1=[]
    for i in yt1:
        lq=[]
        for j in i.split():
            if(changetonum(j)!=-1):
                lq.append(changetonum(j))
        yt_corpus1.append(lq)
    return yt_corpus1

In [ ]:
train11=preprocessing2(train11)
train22=preprocessing2(train22)
train33=preprocessing2(train33)
train44=preprocessing2(train44)

In [ ]:
print(len(cv.vocabulary_))

10000


In [ ]:
import numpy as np
def vectorize_sentences(sentences,dim=10000):
    outputs = np.zeros((len(sentences),dim))
    for i,j in enumerate(sentences):
        for k in j:
            outputs[i,k] = 1
    return outputs

In [ ]:
train_corpus1=vectorize_sentences(train_corpus1)

In [ ]:
test_corpus1=vectorize_sentences(test_corpus1)

In [ ]:
yt_corpus1=vectorize_sentences(yt_corpus1)

In [ ]:
IMDB_corpus1=vectorize_sentences(IMDB_corpus1)

NameError: ignored

In [ ]:
yt_corpus1.shape

(3, 10000)

In [ ]:
train11=vectorize_sentences(train11)
train22=vectorize_sentences(train22)
train33=vectorize_sentences(train33)
train44=vectorize_sentences(train44)

In [ ]:
train['label']

0        0
1        0
2        0
3        0
4        0
        ..
31957    0
31958    0
31959    0
31960    1
31961    0
Name: label, Length: 31962, dtype: int64

## Testing different models and evaluating

In [ ]:
# splitting the training data into train and valid sets

from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(train_corpus1, train['label'], test_size = 0.25, random_state = 42)

print(x_train.shape)
print(x_valid.shape)
print(y_train.shape)
print(y_valid.shape)

In [ ]:
from sklearn.svm import SVC

model = SVC()
model.fit(x_train, y_train)

y_pred = model.predict(x_valid)

print("Training Accuracy :", model.score(x_train, y_train))
print("Validation Accuracy :", model.score(x_valid, y_valid))

# calculating the f1 score for the validation set
print("f1 score :", f1_score(y_valid, y_pred))

# confusion matrix
cm = confusion_matrix(y_valid, y_pred)
print(cm)


In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(x_train, y_train)

y_pred = model.predict(x_valid)

print("Training Accuracy :", model.score(x_train, y_train))
print("Validation Accuracy :", model.score(x_valid, y_valid))

# calculating the f1 score for the validation set
print("f1 score :", f1_score(y_valid, y_pred))

# confusion matrix
cm = confusion_matrix(y_valid, y_pred)
print(cm)


In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(x_train, y_train)

y_pred = model.predict(x_valid)

print("Training Accuracy :", model.score(x_train, y_train))
print("Validation Accuracy :", model.score(x_valid, y_valid))

# calculating the f1 score for the validation set
print("f1 score :", f1_score(y_valid, y_pred))

# confusion matrix
cm = confusion_matrix(y_valid, y_pred)
print(cm)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

model = RandomForestClassifier()
model.fit(x_train, y_train)

y_pred = model.predict(x_valid)

print("Training Accuracy :", model.score(x_train, y_train))
print("Validation Accuracy :", model.score(x_valid, y_valid))

# calculating the f1 score for the validation set
print("F1 score :", f1_score(y_valid, y_pred))

# confusion matrix
cm = confusion_matrix(y_valid, y_pred)
print(cm)



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score


np.concatenate((train11,train22,train33,train44), axis=0)
np.concatenate((train1['1'],train2['1'],train3['0'],train4['0']), axis=0)
model = RandomForestClassifier()
model.fit(train11,train1['1'])

RandomForestClassifier()

In [ ]:
from sklearn.naive_bayes import GaussianNB
gmodel = GaussianNB()
gmodel.fit(train_corpus1,train['label'])

GaussianNB()

In [ ]:
import joblib
joblib.dump(model, 'model.pkl')

['model.pkl']

In [ ]:
joblib.dump(gmodel, 'Gaussmodel.pkl')

['Gaussmodel.pkl']

In [ ]:
model_from_joblib = joblib.load('Gaussmodel.pkl')
sum(model_from_joblib.predict(test_corpus1))

5059

In [ ]:
test_corpus1.shape

(17197, 10000)

In [ ]:
x_train.shape

In [ ]:
x_valid.shape

In [ ]:
381+1674

In [ ]:
sum(train['label'])